<a href="https://colab.research.google.com/github/lkhok22/ML-FinalProject-Walmart-Recruiting---Store-Sales-Forecasting/blob/main/model_experiment_TFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Only install what you need for SARIMAX and logging
!pip install statsmodels wandb --quiet

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import wandb
import os
from datetime import datetime
import warnings
import gc
import psutil
warnings.filterwarnings("ignore")

In [3]:
# Install required libraries
!pip install pandas numpy matplotlib seaborn scikit-learn torch torchvision wandb pyyaml darts --quiet
import wandb
wandb.login(key="eccf2c915699fc032ad678daf0fd4b5ac60bf87c")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abakh22 (abakh22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
# Mount Google Drive and extract data
from google.colab import drive
import zipfile
import os
drive.mount('/content/drive')
zip_path = '/content/drive/MyDrive/ML-FinalProject/data.zip'
extract_to = '/content/walmart_data/'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
for file_name in os.listdir(extract_to):
    if file_name.endswith('.zip'):
        with zipfile.ZipFile(os.path.join(extract_to, file_name), 'r') as zip_ref:
            zip_ref.extractall(extract_to)
print("✅ Extracted files:", os.listdir(extract_to))

Mounted at /content/drive
✅ Extracted files: ['test.csv.zip', 'stores.csv', 'sampleSubmission.csv', 'features.csv', 'test.csv', 'train.csv.zip', 'train.csv', 'sampleSubmission.csv.zip', 'features.csv.zip']


In [6]:
# Load data with optimized dtypes
train = pd.read_csv('/content/walmart_data/train.csv')
features = pd.read_csv('/content/walmart_data/features.csv')
stores = pd.read_csv('/content/walmart_data/stores.csv')
test = pd.read_csv('/content/walmart_data/test.csv')

# Merge and keep only necessary columns
df = pd.merge(train, features, on=['Store', 'Date'], how='left')
df = pd.merge(df, stores, on='Store', how='left')
df = df.drop(columns=['IsHoliday_x']).rename(columns={'IsHoliday_y': 'IsHoliday'})
train_merged = df[['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
         'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'MarkDown1', 'MarkDown2',
         'MarkDown3', 'MarkDown4', 'MarkDown5']].sort_values(by=['Store', 'Dept', 'Date'])

test_df = pd.merge(test, features, on=['Store', 'Date'], how='left')
test_df = pd.merge(test_df, stores, on='Store', how='left')
test_df = test_df.drop(columns=['IsHoliday_x']).rename(columns={'IsHoliday_y': 'IsHoliday'})
test_merged = test_df[['Store', 'Dept', 'Date', 'IsHoliday', 'Temperature',
                   'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'MarkDown1',
                   'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].sort_values(by=['Store', 'Dept', 'Date'])

# Convert Date to datetime
train_merged['Date'] = pd.to_datetime(train_merged['Date'])
test_merged['Date'] = pd.to_datetime(test_merged['Date'])

print("✅ Merged train shape:", train_merged.shape)
print("✅ Merged test shape:", test_merged.shape)

✅ Merged train shape: (421570, 15)
✅ Merged test shape: (115064, 14)


In [9]:
# Inspect column names to diagnose the issue
print("✅ Stores columns:", stores.columns.tolist())
print("✅ Train merged columns:", train_merged.columns.tolist())
print("✅ Test merged columns:", test_merged.columns.tolist())

# Handle missing values
# MarkDown1-5: Fill NA with 0 (no markdown event)
for col in ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']:
    train_merged[col] = train_merged[col].fillna(0)
    test_merged[col] = test_merged[col].fillna(0)

# Other numerical features: Fill NA with median
for col in ['CPI', 'Unemployment']:
    train_merged[col] = train_merged[col].fillna(train_merged[col].median())
    test_merged[col] = test_merged[col].fillna(test_merged[col].median())

# Encode categorical variables
# Check if 'Type' exists, and use the correct column name
type_col = 'Type' if 'Type' in train_merged.columns else None
if type_col:
    train_merged[type_col] = train_merged[type_col].astype('category').cat.codes
    test_merged[type_col] = test_merged[type_col].astype('category').cat.codes
else:
    print("⚠️ Column 'Type' not found in merged data. Skipping encoding for 'Type'.")
    # Optionally, add a placeholder column if needed
    train_merged['Type'] = 0
    test_merged['Type'] = 0

# Encode IsHoliday
train_merged['IsHoliday'] = train_merged['IsHoliday'].astype(int)
test_merged['IsHoliday'] = test_merged['IsHoliday'].astype(int)

print("✅ Missing values handled and categoricals encoded")

✅ Stores columns: ['Store', 'Type', 'Size']
✅ Train merged columns: ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
✅ Test merged columns: ['Store', 'Dept', 'Date', 'IsHoliday', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
⚠️ Column 'Type' not found in merged data. Skipping encoding for 'Type'.
✅ Missing values handled and categoricals encoded


In [ ]:
from darts import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries

# Create TimeSeries objects for each store-department
def create_timeseries(df, target_col='Weekly_Sales', is_train=True, min_length=16):
    series_dict = {}
    for (store, dept), group in df.groupby(['Store', 'Dept']):
        group = group.sort_values('Date')
        # Ensure group has enough data
        if len(group) < min_length:
            print(f"⚠️ Skipping Store {store}, Dept {dept}: Only {len(group)} time steps (min required: {min_length})")
            continue

        # Target series (only for training data)
        target_series = None
        if is_train:
            if target_col in group.columns:
                target_series = TimeSeries.from_dataframe(
                    group, time_col='Date', value_cols=[target_col], freq='W-FRI'
                )
            else:
                print(f"⚠️ Target column '{target_col}' not found for Store {store}, Dept {dept}. Skipping.")
                continue

        # Past covariates (e.g., Temperature, Fuel_Price, MarkDowns, CPI, Unemployment, Type, IsHoliday)
        past_cov_columns = ['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
                           'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'IsHoliday']
        # Filter columns that exist in the DataFrame
        past_cov_columns = [col for col in past_cov_columns if col in group.columns]
        past_cov = TimeSeries.from_dataframe(
            group, time_col='Date', value_cols=past_cov_columns, freq='W-FRI'
        )

        # Future covariates (known in advance: IsHoliday, Date-based features)
        future_cov = TimeSeries.from_dataframe(
            group, time_col='Date', value_cols=['IsHoliday'], freq='W-FRI'
        )

        # Add time-based features (month one-hot, week as numerical)
        time_index = TimeSeries.from_dataframe(group, time_col='Date', value_cols=['IsHoliday'], freq='W-FRI')
        month_features = datetime_attribute_timeseries(
            time_index, attribute='month', one_hot=True
        )
        week_features = datetime_attribute_timeseries(
            time_index, attribute='week', one_hot=False  # Use numerical week number
        )
        time_features = month_features.concatenate(week_features, axis=1)
        future_cov = future_cov.concatenate(time_features, axis=1)

        # Store series
        series_dict[(store, dept)] = {
            'target': target_series,
            'past_cov': past_cov,
            'future_cov': future_cov
        }
    return series_dict

# Create training and test series
train_series = create_timeseries(train_merged, target_col='Weekly_Sales', is_train=True, min_length=16)
test_series = create_timeseries(test_merged, is_train=False, min_length=1)  # Test set needs at least 1 time step

print(f"✅ Created {len(train_series)} training time series")
print(f"✅ Created {len(test_series)} test time series")

In [ ]:
from darts.models import TFTModel
from darts.metrics import mae
import torch
import threadpoolctl
import numpy as np

# Suppress threadpoolctl warnings
threadpoolctl.threadpool_limits(1, "blas")  # Limit to 1 thread to avoid dlopen issues

# Initialize wandb
wandb.init(project="walmart_sales_forecasting", name="tft_simple_model")

# Define WMAE metric
def wmae(y_true, y_pred, is_holiday):
    weights = 5 * is_holiday + (1 - is_holiday)
    return np.mean(weights * np.abs(y_true - y_pred))

# TFT model configuration
model = TFTModel(
    input_chunk_length=12,  # Look back 12 weeks
    output_chunk_length=4,  # Predict 4 weeks ahead
    hidden_size=16,         # Small for efficiency
    lstm_layers=1,
    num_attention_heads=4,
    dropout=0.1,
    batch_size=32,
    n_epochs=10,            # Adjust based on performance
    add_relative_index=True,
    random_state=42,
    pl_trainer_kwargs={"accelerator": "gpu" if torch.cuda.is_available() else "cpu"}
)

# Train model on all series
train_targets = [series['target'] for series in train_series.values() if series['target'] is not None]
train_past_covs = [series['past_cov'] for series in train_series.values() if series['target'] is not None]
train_future_covs = [series['future_cov'] for series in train_series.values() if series['target'] is not None]

# Log number of valid series
wandb.log({"num_training_series": len(train_targets)})

model.fit(
    series=train_targets,
    past_covariates=train_past_covs,
    future_covariates=train_future_covs,
    verbose=True
)

# Log training completion
wandb.log({"status": "Training completed"})

In [ ]:
# Generate predictions
predictions = {}
for (store, dept), series in test_series.items():
    pred = model.predict(
        n=len(series['target']),
        series=series['target'],
        past_covariates=series['past_cov'],
        future_covariates=series['future_cov']
    )
    predictions[(store, dept)] = pred

# Format submission
submission = []
for (store, dept), pred in predictions.items():
    dates = pred.time_index
    values = pred.values().flatten()
    for date, value in zip(dates, values):
        id_str = f"{store}_{dept}_{date.strftime('%Y-%m-%d')}"
        submission.append({'Id': id_str, 'Weekly_Sales': max(0, value)})  # Ensure non-negative sales

submission_df = pd.DataFrame(submission)
submission_df.to_csv('/content/walmart_submission.csv', index=False)

# Log submission file to wandb
wandb.save('/content/walmart_submission.csv')
print("✅ Submission file created: walmart_submission.csv")

In [ ]:
# Validation (optional, if you want to compute WMAE on a validation split)
# Split train data into train/validation (last 4 weeks for validation)
val_targets = []
val_preds = []
val_holidays = []
for (store, dept), series in train_series.items():
    train_split = series['target'][:-4]
    val_split = series['target'][-4:]
    past_cov_split = series['past_cov'][:-4]
    future_cov_split = series['future_cov']
    pred = model.predict(
        n=4,
        series=train_split,
        past_covariates=past_cov_split,
        future_covariates=future_cov_split
    )
    val_targets.append(val_split.values().flatten())
    val_preds.append(pred.values().flatten())
    val_holidays.append(series['future_cov']['IsHoliday'][-4:].values().flatten())

# Compute WMAE
val_targets = np.concatenate(val_targets)
val_preds = np.concatenate(val_preds)
val_holidays = np.concatenate(val_holidays)
wmae_score = wmae(val_targets, val_preds, val_holidays)
wandb.log({"validation_wmae": wmae_score})
print(f"✅ Validation WMAE: {wmae_score}")